In [ ]:
import os

is_local = False

onetoone_function = "onetoone"
divide_function = "divide"
mergepairs_function = "mergepairs"

if is_local:
    url = "http://localhost:7071/api/"
    
else:
    url = "https://mappingexample.azurewebsites.net/api/"
    
    
onetoone_url = url + onetoone_function
divide_url = url + divide_function
mergepairs_url = url + mergepairs_function




In [ ]:
import requests
import json


data = {
    "values": [
        {
            "recordId": "record1",
            "data": { 
                "function_name_input": "inputstring1",
            }
        },  
        {
            "recordId": "record2",
            "data":  { 
                "function_name_input": "inputstring2",
            }
        },
    ]
}



In [ ]:
response = requests.post(onetoone_url, json=data)
print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4))
except requests.exceptions.JSONDecodeError:
    print(response.text)

In [ ]:
response = requests.post(divide_url, json=data)
print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4))
except requests.exceptions.JSONDecodeError:
    print(response.text)

In [ ]:
from openai import AzureOpenAI

system_prompt = """
You are an AI assistance that extracts text from the image. You are especially good at extracting tables.
Start your response with the page number of the image. 
Example Page:

Page 123

Monthly Savings
| Month    | Savings |Details      |
| -------- | ------- |------------ |
| January  | $250    | for holiday |
| February | $80     | pension     |
| March    | $420    | new cat     |

Savings were significantly lower in February. This is surprising because it is a short month and contributing to pension shuld be a priority.
"""


client = AzureOpenAI(
    azure_endpoint=os.environ['OPENAI_API_ENDPOINT'],
    api_key=os.environ['OPENAI_API_KEY'],
    api_version='2023-05-15',
    )

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Extract text from the image"},
                {
                    "type": "image_url",
                    "image_url": {"url": data["values"][0]["data"]["image_sas_urls"][0]},
                },
            ],
        },
    ],
    max_tokens=2000,
    temperature=0.0,
)

print(response.choices[0].message.content)

In [ ]:
# Extract the page number from the markdown string
markdown = """     Pageer 43

page 456
    
    Monthly Savings
    | Month    | Savings |Details      |
    | -------- | ------- |------------ |
    | January  | $250    | for holiday |
    | February | $80     | pension     |
    | March    | $420    | new cat     |
    
    Savings were significantly lower in February. This is surprising because it is a short month and contributing to pension shuld be a priority.
    """
    
page_number = markdown.split('\n')[0].replace("Page ", "").strip()
print(page_number)

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobSasPermissions, generate_blob_sas, BlobClient

pdf_url=pdf_url1+"?"+pdf_sas1
print(pdf_url)
blob_client = BlobClient.from_blob_url(pdf_url)
pdf_bytes = blob_client.download_blob().readall()

In [ ]:
import datetime

blob_connection_string = os.environ["BLOB_STORAGE_ACCOUNT_CONNECTION_STRING"]
blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)
container_client = blob_service_client.get_container_client("customskillimages")


def get_account_key_from_connection_string(connection_string):
    parts = connection_string.split(';')
    for part in parts:
        if part.startswith('AccountKey='):
            return part[len('AccountKey='):]
    return None



account_key = get_account_key_from_connection_string(blob_connection_string)
print(account_key)
sas_token = generate_blob_sas(
    account_key=account_key,
    account_name=blob_service_client.account_name,
    container_name=container_client.container_name,
    blob_name=blob_client.blob_name,
    permission=BlobSasPermissions(read=True),
    expiry=datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=1)
)